<a href="https://colab.research.google.com/github/victormuelacarriles/GoogleColab/blob/main/Buscando_NTLM2_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Probando Hascat desde los servidores de google**
Esta prueba se basa en modificaciones del proyecto Cloudtopolis (https://github.com/JoelGMSec/Cloudtopolis).

En lugar de buscar multiples sesiones, simplemente se quiere ejecutar una sesión para resolver hashes (en concreto de NTML).
Se ha divido en varias secciones:

1º) Preparar entorno
2º) Preparar fichero con hashes
3º) Comprobar velocidad de trabajo
4º) Busqueda de hashes

# 1º)Preparación de entorno
Se actualizan paquetes e instalan hascat y utilidades para comprobar la tarjeta gráfica. 

In [ ]:
!echo -e "\n\e[1;34mPreparación del entorno\e[0m (tardara un par de minutos mínimo). Esta parte sólo hace falta ejecutarla una vez."               
!echo "Buscando paquetes nuevos..."
!apt update -y -q > /dev/null 2>&1
!echo "Actualizando paquetes..."
!apt upgrade -y -q > /dev/null 2>&1
!echo "Instalando hashcat ..."
!apt install hashcat -y -q > /dev/null 2>&1
!echo "Instalando utilidades de pci ..."
!apt install pciutils -y -q > /dev/null 2>&1
Rockyoutxt = !ls /content/rockyou.txt
if not "/content/rockyou.txt" in Rockyoutxt:
        # Downloading Rockyou
        !echo -e "\e[0mDiccionario Rockyou no encontrado!"
        !echo -e "\n\e[34;1mDescargando Rockyou.."
        !wget --quiet https://download.weakpass.com/wordlists/90/rockyou.txt.gz
        !wget --quiet https://raw.githubusercontent.com/hashcat/hashcat/master/rules/rockyou-30000.rule

        # Extracting wordlist
        !echo -e "\e[0mExtrayendo Rockyou.."
        !gunzip -f rockyou.txt.gz > /dev/null 2>&1


!echo -e "\e[1;34mListo para trabajar. Asegúrate de tener un fichero 'ficheroHASHES.txt' creado antes de empezar la explotación\e[0m"


# 2º) Preparación de fichero con hashes
Se crea el fichero "ficheroHASHES.txt" con unos cuantos hashes de ejemplo de tipo NTLM. 
Se establece la variable tipoHASH a 1000 (para indicar el tipo NTLM). Para otros tipos ver la ayuda de hashcat.


In [ ]:
!rm ficheroHASHES.txt 2>/dev/null
tipoHASH= 5600   #tipo NTLM2   #https://www.browserling.com/tools/ntlm-hash
#tipoHASH=1000 #NTLM

#!echo _ >>ficheroHASHES.txt # _ en NTLM
#!echo _ >>ficheroHASHES.txt # _ en NTLM

!echo >ficheroHASHES.txt
#Ejemplo: usuario "DOMINIO\administrador" con el hash de su password 'usuario@1'
!echo Administrador::DOMINIO:eb63ebe969b740bc:E48E1EB16045F5823B64C4A368B25D07:01010000000000001C7233C170DFD601626BF2E5CB9C1D54000000000200060053004D0042000100160053004D0042002D0054004F004F004C004B00490054000400120073006D0062002E006C006F00630061006C000300280073006500720076006500720032003000300033002E0073006D0062002E006C006F00630061006C000500120073006D0062002E006C006F00630061006C000800300030000000000000000000000000300000A15D9019357C09267DC86DBE72CE15A4B63D58BEAE68365C81D90857C8E641D70A001000000000000000000000000000000000000900120048005400540050002F0077007000610064000000000000000000>>ficheroHASHES.txt
!echo -e "\e[1;34mCreado fichero de hashes tipo "$tipoHASH". Contenido:\e[0m"
!cat ficheroHASHES.txt



#3º) Comprobación de velocidad para búsquedas de tipo 1000 (NTLM).
Referencia Tarjeta gráfica básica:   GeForce GT 1030, 1660/2048 MB, 3MCU
Speed.:  6717.8 MH/s (29.78ms) @ Accel:64 Loops:1024 Thr:1024 Vec:8


In [ ]:
GPU_CARD = !lspci | grep NVIDIA
if not GPU_CARD:
    !echo -e "\n\e[31;1m[!] No se ha encontrado una GPU NVIDA! (Revisar: Editar->Configuración del cuaderno->Aceleración por Hardware->GPU\e[34;1m"
    !lspci
else:
    !echo -e "\e[1;34mEncontrada tarjeta gráfica NVIDIA\e[0m"
    !echo -e "\e[1;34mTest de Velocidad para tipo 1000\e[0m" 
    !hashcat --benchmark -m1000 > TestDeVelocidad.txt 2>/dev/null
    !cat TestDeVelocidad.txt|grep Device
    !cat TestDeVelocidad.txt|grep Speed
   

# 4º) Búsqueda de Hashes

## 4a) Ataque de diccionario

In [ ]:
#Ejemplo de ejecución
if not tipoHASH:
  tipoHASH=5600  #NTLMv2
  #tipoHASH=1000 #NTLM
#Ataque de diccionario basado en rockyou.txt con reglas de variación
!hashcat -a 0 -m $tipoHASH --potfile-path ./potfile.txt ficheroHASHES.txt ./rockyou.txt -r ./rockyou-30000.rule -O 


## 4b) Ataque de fuerza bruta incremental

In [ ]:
#Ejemplo de ejecución
if not tipoHASH:
  tipoHASH=5600  #NTLMv2
  #tipoHASH=1000 #NTLM

#!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt  --increment ficheroHASHES.txt '?a?a?a?a?a?a?a?a?a' -O


!echo -e "\n\e[1;34mAtaque de hasta 6 '?a' (20min aprox) \e[0m (?a= cualquier digito, letra mayuscula o minúscula y simbolos estandar) "
!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt  --increment ficheroHASHES.txt '?a?a?a?a?a?a' -O    #hasta 2 minutos -> 13 minutos
#!hashcat -m$tipoHASH -a3 -w4 -O --potfile-path ./potfile.txt  ficheroHASHES.txt '?a?a?a?a?a?a?a' -O   # 6 13min
 # ? | Charset
 #===+=========
 # l | abcdefghijklmnopqrstuvwxyz
 # u | ABCDEFGHIJKLMNOPQRSTUVWXYZ
 # d | 0123456789
 # h | 0123456789abcdef
 # H | 0123456789ABCDEF
 # s |  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
 # a | ?l?u?d?s
 # b | 0x00 - 0xff

#!hashcat -m 5600 -a 3 --potfile-path ./potfile.txt ficheroHASHES.txt   #https://medium.com/@petergombos/lm-ntlm-net-ntlmv2-oh-my-a9b235c58ed4

#Ayuda de hashcat

In [ ]:
#Para ver la ayuda de hascat completa:
!hashcat --help